# SERVER

In [ ]:
from helper.lrmodel import LRModel
import phe as paillier
import json

In [ ]:
def getData():  # TODO: replace with fission serverless function call
    with open('keys/encrypted_data.json', 'r') as file:
        d = json.load(file)
    data = json.loads(d)
    return data


def computeData():
    data = getData()
    mycoef = LRModel().getCoef()

    # #! just to test right calculation
    # client_data = [24, 4, 6, 1]
    # print("The calculation result of server is:\n")
    # print(sum([client_data[i]*mycoef[i] for i in range(len(client_data))]))
    # #! just to test right calculation
    
    pk = data['public_key']
    pubkey = paillier.PaillierPublicKey(n=int(pk['n']))
    enc_nums_rec = [paillier.EncryptedNumber(
        pubkey, int(x[0], int(x[1]))) for x in data['values']]
    results = sum([mycoef[i]*enc_nums_rec[i] for i in range(len(mycoef))])
    return results, pubkey  # encrypted result of aggregation


def serializeData():
    results, pubkey = computeData()
    encrypted_data = {}
    encrypted_data['pubkey'] = {'n': pubkey.n}
    encrypted_data['values'] = (str(results.ciphertext()), results.exponent)
    serialized = json.dumps(encrypted_data)
    return serialized

In [ ]:
datafile=serializeData()
with open('keys/cyphered_computation.json', 'w') as file:
    json.dump(datafile, file)